> **⚠️ EN COURS DE CONSTRUCTION**
>
> Ce notebook est en cours de finalisation. Merci de ne pas le consulter pour l'instant.

# TP 03 - Multi-Head Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Comprendre et implémenter le Multi-Head Attention

---

## Objectifs pédagogiques

À la fin de ce TP, vous serez capable de :
1. Expliquer pourquoi plusieurs têtes d'attention sont utiles
2. Implémenter le Multi-Head Attention from scratch
3. Visualiser ce que chaque tête apprend
4. Comprendre le lien avec les Transformers

---

## Prérequis

Ce TP suppose que vous avez complété le **TP 02 - Mécanisme d'Attention** où vous avez :
- Implémenté le Scaled Dot-Product Attention
- Compris les concepts de Query, Key, Value
- Visualisé l'attention sur un vrai modèle

Ici, nous allons voir comment **combiner plusieurs têtes d'attention** pour capturer différents types de relations.

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

## 1. Rappel : Single-Head Attention

Reprenons notre fonction d'attention du TP précédent.

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (..., seq_len, d_k)
        K: Keys, shape (..., seq_len, d_k)
        V: Values, shape (..., seq_len, d_v)
        mask: Masque optionnel
    
    Returns:
        output, attention_weights
    """
    d_k = K.shape[-1]
    scores = Q @ K.transpose(-2, -1) / math.sqrt(d_k)
    
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    
    attention_weights = F.softmax(scores, dim=-1)
    output = attention_weights @ V
    
    return output, attention_weights

---

## 2. Pourquoi Multi-Head ?

### Le problème avec une seule tête

Une seule tête d'attention calcule **une** représentation des relations entre mots.

Mais dans une phrase, il y a **plusieurs types de relations** :
- Relations syntaxiques (sujet-verbe)
- Relations sémantiques (sens)
- Relations de proximité
- etc.

### La solution : plusieurs têtes

Chaque tête peut apprendre à détecter un type de relation différent !

**Analogie** : C'est comme avoir plusieurs experts qui analysent une phrase sous différents angles, puis combinent leurs analyses.

In [ ]:
# Illustration : différentes têtes peuvent capturer différentes relations
phrase = ["Le", "chat", "noir", "mange", "la", "souris"]

# Tête 1 : Relations syntaxiques (sujet-verbe)
attention_syntaxe = torch.tensor([
    [0.3, 0.5, 0.1, 0.05, 0.03, 0.02],  # "Le" → "chat"
    [0.1, 0.3, 0.1, 0.4, 0.05, 0.05],   # "chat" → "mange"
    [0.1, 0.6, 0.2, 0.05, 0.03, 0.02],  # "noir" → "chat"
    [0.05, 0.5, 0.05, 0.2, 0.1, 0.1],   # "mange" → "chat"
    [0.02, 0.03, 0.02, 0.03, 0.3, 0.6], # "la" → "souris"
    [0.02, 0.1, 0.02, 0.4, 0.06, 0.4],  # "souris" → "mange"
])

# Tête 2 : Relations de proximité
attention_proximite = torch.tensor([
    [0.5, 0.4, 0.08, 0.01, 0.005, 0.005],
    [0.3, 0.4, 0.25, 0.04, 0.005, 0.005],
    [0.1, 0.35, 0.35, 0.15, 0.03, 0.02],
    [0.02, 0.1, 0.3, 0.35, 0.18, 0.05],
    [0.01, 0.02, 0.05, 0.25, 0.4, 0.27],
    [0.005, 0.01, 0.02, 0.1, 0.35, 0.515],
])

# Visualisation côte à côte
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (attn, title) in enumerate([
    (attention_syntaxe, "Tête 1 : Relations syntaxiques"),
    (attention_proximite, "Tête 2 : Proximité")
]):
    ax = axes[idx]
    im = ax.imshow(attn, cmap='Blues')
    ax.set_xticks(range(6))
    ax.set_xticklabels(phrase, rotation=45)
    ax.set_yticks(range(6))
    ax.set_yticklabels(phrase)
    ax.set_title(title)
    plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

---

## 3. Architecture Multi-Head Attention

### Schéma

```
Input (seq_len, embed_dim)
        ↓
   ┌────┴────┬────────┬────────┐
   ↓         ↓        ↓        ↓
 Head 1   Head 2   Head 3   Head 4   (chaque tête a sa propre projection Q, K, V)
   ↓         ↓        ↓        ↓
   └────┬────┴────────┴────────┘
        ↓
    Concat
        ↓
   Linear (projection de sortie)
        ↓
Output (seq_len, embed_dim)
```

### Dimensions

- **embed_dim** : Dimension des embeddings (ex: 512)
- **num_heads** : Nombre de têtes (ex: 8)
- **d_k = embed_dim / num_heads** : Dimension par tête (ex: 64)

Chaque tête travaille avec une dimension réduite, puis on concatène.

---

## 4. Implémentation étape par étape

### Étape 1 : Projections Q, K, V

In [ ]:
# Configuration
batch_size = 2
seq_len = 6
embed_dim = 32
num_heads = 4
d_k = embed_dim // num_heads  # 32 / 4 = 8

print(f"embed_dim: {embed_dim}")
print(f"num_heads: {num_heads}")
print(f"d_k (dim par tête): {d_k}")

# Input
x = torch.randn(batch_size, seq_len, embed_dim)
print(f"\nInput shape: {x.shape}")

In [ ]:
# Créer les projections
# On projette vers embed_dim (pas num_heads * d_k, c'est la même chose)
W_q = nn.Linear(embed_dim, embed_dim)
W_k = nn.Linear(embed_dim, embed_dim)
W_v = nn.Linear(embed_dim, embed_dim)

# Projeter
Q = W_q(x)  # (batch, seq_len, embed_dim)
K = W_k(x)
V = W_v(x)

print(f"Q shape après projection: {Q.shape}")

### Étape 2 : Reshape pour séparer les têtes

On doit transformer `(batch, seq_len, embed_dim)` en `(batch, num_heads, seq_len, d_k)`

In [ ]:
# ============================================
# EXERCICE 1 : Reshape pour multi-head
# ============================================

# Transformer (batch, seq_len, embed_dim) en (batch, num_heads, seq_len, d_k)
# Indice : utilisez view() et transpose()

def split_heads(x, num_heads):
    """
    Reshape (batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)
    """
    batch_size, seq_len, embed_dim = x.shape
    d_k = embed_dim // num_heads
    
    # TODO: Implémenter le reshape en 2 étapes
    # - Séparer embed_dim en (num_heads, d_k)
    # - Réorganiser pour avoir num_heads en position 1
    
    x = None  # À compléter
    
    return x

# Test (décommentez après avoir complété l'exercice)
# Q_heads = split_heads(Q, num_heads)
# print(f"Q shape après split: {Q_heads.shape}")  # Attendu: (2, 4, 6, 8)

### Étape 3 : Attention par tête

### Étape 4 : Concaténer les têtes

In [ ]:
# ============================================
# EXERCICE 2 : Concat des têtes
# ============================================

def concat_heads(x):
    """
    Reshape (batch, num_heads, seq_len, d_k) -> (batch, seq_len, embed_dim)
    C'est l'inverse de split_heads
    """
    batch_size, num_heads, seq_len, d_k = x.shape
    embed_dim = num_heads * d_k
    
    # TODO: Implémenter le reshape inverse
    # Indice : inverse des opérations de split_heads
    # N'oubliez pas .contiguous() si nécessaire
    
    x = None  # À compléter
    
    return x

# Test (décommentez après avoir complété les exercices 1 et 2)
# Q_heads = split_heads(Q, num_heads)
# K_heads = split_heads(K, num_heads)
# V_heads = split_heads(V, num_heads)
# attn_output, attn_weights = scaled_dot_product_attention(Q_heads, K_heads, V_heads)
# concat_output = concat_heads(attn_output)
# print(f"Output après concat: {concat_output.shape}")  # Attendu: (2, 6, 32)

### Étape 5 : Projection de sortie

In [ ]:
# Projection finale
W_o = nn.Linear(embed_dim, embed_dim)

final_output = W_o(concat_output)
print(f"Final output shape: {final_output.shape}")  # (2, 6, 32)

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Attention.
    
    Args:
        embed_dim: Dimension des embeddings
        num_heads: Nombre de têtes d'attention
    """
    
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        
        assert embed_dim % num_heads == 0, "embed_dim doit être divisible par num_heads"
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.d_k = embed_dim // num_heads
        
        # TODO: Créer les 4 projections linéaires
        self.W_q = None  # nn.Linear(embed_dim, embed_dim)
        self.W_k = None
        self.W_v = None
        self.W_o = None
    
    def split_heads(self, x):
        """(batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)"""
        batch_size, seq_len, _ = x.shape
        # TODO: Implémenter
        return None
    
    def concat_heads(self, x):
        """(batch, num_heads, seq_len, d_k) -> (batch, seq_len, embed_dim)"""
        batch_size, _, seq_len, _ = x.shape
        # TODO: Implémenter
        return None
    
    def forward(self, x, mask=None):
        """
        Args:
            x: Input, shape (batch, seq_len, embed_dim)
            mask: Masque optionnel
        
        Returns:
            output: (batch, seq_len, embed_dim)
            attention_weights: (batch, num_heads, seq_len, seq_len)
        """
        # TODO: Implémenter le forward
        # 1. Projeter x en Q, K, V
        Q = None
        K = None
        V = None
        
        # 2. Split en têtes
        Q = None  # self.split_heads(Q)
        K = None
        V = None
        
        # 3. Attention
        attn_output, attn_weights = None, None  # scaled_dot_product_attention(Q, K, V, mask)
        
        # 4. Concat
        concat_output = None  # self.concat_heads(attn_output)
        
        # 5. Projection de sortie
        output = None  # self.W_o(concat_output)
        
        return output, attn_weights

In [ ]:
# Test de votre implémentation
mha = MultiHeadAttention(embed_dim=32, num_heads=4)

x = torch.randn(2, 6, 32)  # batch=2, seq=6, embed=32
output, weights = mha(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # Attendu: (2, 6, 32)
print(f"Weights shape: {weights.shape}")  # Attendu: (2, 4, 6, 6)

---

## 6. Visualisation des têtes sur un vrai modèle

Voyons ce que les différentes têtes capturent sur un modèle **réellement entraîné**.

In [ ]:
# Charger DistilBERT (comme au TP 02)
!pip install transformers -q

from transformers import AutoModel, AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)
model.eval()

# Phrase de test
phrase = "The cat sat on the mat because it was tired"
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Extraire les attentions de la couche 5 (12 têtes)
layer = 4  # Couche 5 (index 0-5)
attention = outputs.attentions[layer][0]  # (num_heads, seq_len, seq_len)

print(f"Phrase: {phrase}")
print(f"Tokens: {tokens}")
print(f"Shape attention couche {layer+1}: {attention.shape}")  # (12, 11, 11)

In [ ]:
# Visualiser 4 têtes différentes de la même couche
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Sélectionner 4 têtes intéressantes
heads_to_show = [1, 2, 5, 10]  # Différentes têtes

for idx, head in enumerate(heads_to_show):
    ax = axes[idx // 2, idx % 2]
    w = attention[head].numpy()
    
    im = ax.imshow(w, cmap='Blues')
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=9)
    ax.set_yticks(range(len(tokens)))
    ax.set_yticklabels(tokens, fontsize=9)
    ax.set_title(f"Tête {head + 1}", fontsize=12)
    plt.colorbar(im, ax=ax)

plt.suptitle(f"Différentes têtes de la couche {layer+1} - Chaque tête capture des relations différentes", fontsize=13)
plt.tight_layout()
plt.show()

# Analyse spécifique : que regarde "it" selon différentes têtes ?
it_index = tokens.index("it")
print(f"\n{'='*60}")
print(f"Que regarde le pronom 'it' selon chaque tête ?")
print(f"{'='*60}")

for head in heads_to_show:
    print(f"\n--- Tête {head + 1} ---")
    weights = attention[head, it_index].numpy()
    top_indices = weights.argsort()[-3:][::-1]  # Top 3
    for i in top_indices:
        bar = "█" * int(weights[i] * 20)
        print(f"  {tokens[i]:10} {weights[i]:.2f} {bar}")

**Observation** : Chaque tête a appris à capturer des **relations différentes** :
- Certaines têtes se concentrent sur la **coréférence** ("it" → "cat")
- D'autres sur la **proximité** (mots voisins)
- D'autres sur la **syntaxe** (sujet-verbe)
- Certaines regardent le token **[CLS]** (représentation globale)

C'est exactement ce qu'on voulait ! Multi-head = **plusieurs experts** qui analysent la phrase sous différents angles.

---

## 7. Comparaison avec PyTorch nn.MultiheadAttention

In [ ]:
# PyTorch fournit une implémentation optimisée
mha_pytorch = nn.MultiheadAttention(embed_dim=32, num_heads=4, batch_first=True)

x = torch.randn(2, 6, 32)

# Pour nn.MultiheadAttention, on passe query, key, value séparément
# En self-attention, query = key = value = x
output_pytorch, weights_pytorch = mha_pytorch(x, x, x)

print(f"Output shape (PyTorch): {output_pytorch.shape}")
print(f"Weights shape (PyTorch): {weights_pytorch.shape}")

---

## 8. Exercice de synthèse : Nombre de paramètres

Calculons le nombre de paramètres dans notre MultiHeadAttention.

In [ ]:
# ============================================
# EXERCICE 4 : Calcul des paramètres
# ============================================

embed_dim = 512
num_heads = 8

# Combien de paramètres dans :
# - W_q : Linear(embed_dim, embed_dim) = embed_dim * embed_dim + embed_dim (weights + bias)
# - W_k : idem
# - W_v : idem
# - W_o : idem

params_per_linear = None  # TODO: Calculer
total_params = None  # TODO: 4 * params_per_linear

print(f"Paramètres par couche linéaire: {params_per_linear:,}")
print(f"Total paramètres MHA: {total_params:,}")

In [ ]:
# Vérification
mha_test = MultiHeadAttention(embed_dim=512, num_heads=8)
real_params = sum(p.numel() for p in mha_test.parameters())
print(f"Vérification PyTorch: {real_params:,} paramètres")

# ============================================
# EXERCICE 4 : Calcul des paramètres
# ============================================

embed_dim = 512
num_heads = 8

# Combien de paramètres dans notre MultiHeadAttention ?
# Rappel : nn.Linear(in, out) a (in * out + out) paramètres (weights + bias)
#
# Indice : Comptez les paramètres de chaque couche linéaire (W_q, W_k, W_v, W_o)

params_per_linear = None  # TODO: Calculer pour une couche Linear(embed_dim, embed_dim)
total_params = None  # TODO: Calculer le total

print(f"Paramètres par couche linéaire: {params_per_linear:,}")
print(f"Total paramètres MHA: {total_params:,}")

---

## 10. Pour aller plus loin : Cross-Attention

> **Note** : Le cross-attention sera abordé en détail dans les **projets** (Mini-GPT, RAG).

**Self-attention** (ce qu'on a fait) : Q, K, V viennent de la **même** source.

**Cross-attention** : Q vient d'une source, K et V d'une **autre** source.
- Utilisé en **traduction** : le décodeur (français) "interroge" l'encodeur (anglais)
- Utilisé dans les **modèles génératifs** avec contexte externe (RAG)

```
Self-attention:     x ──► Q, K, V     (même source)
Cross-attention:    x_dec ──► Q       (une source)
                    x_enc ──► K, V    (autre source)
```

Dans les TP 2-4, nous utilisons uniquement la **self-attention** (encodeur). Le cross-attention sera exploré dans les projets avec les architectures génératives.

---

## 11. Mini-projet : Classifier les Pokémon par type

> **🏠 BONUS / DEVOIR MAISON**
> 
> Cette section est **optionnelle** et ne fait pas partie du TP en session.
> Elle est proposée pour les étudiants qui souhaitent approfondir à la maison.

Dans ce projet, vous allez **fine-tuner CamemBERT** (un modèle BERT français) pour classifier des articles Pokémon par type (Feu, Eau, Plante...), puis **analyser les têtes d'attention** pour comprendre ce que le modèle a appris.

### Objectifs pédagogiques

1. Comprendre le **fine-tuning** d'un modèle pré-entraîné
2. Observer comment les **têtes d'attention changent** après entraînement
3. Analyser si certaines têtes se **spécialisent** sur des patterns spécifiques

### Structure du projet

| Partie | Contenu | Difficulté |
|--------|---------|------------|
| **Partie 1** | Fine-tuning CamemBERT pour classification | ⭐⭐ |
| **Partie 2** | Visualisation de l'attention avant/après | ⭐⭐ |
| **Partie 3** | Analyse des têtes spécialisées | ⭐⭐⭐ |

---

### Partie 1 : Fine-tuning CamemBERT

#### 1.1 Chargement du dataset Pokémon